In [74]:
import os
import sys
import math
import numpy as np
import pandas as pd

In [75]:
with open('2A.txt', mode = 'r') as file:
    Lines = file.readlines()

In [76]:
edges = []
x = 0
for line in Lines:
    if x <= 1:
        x = x + 1
        if x == 1:
            label1, num = line.split(' : ')
            num = int(num)
            rows, cols = (num, num)
            mat = [[0 for i in range(cols)] for j in range(rows)]
    else:
        edges.append(list(map(int, line.split(', '))))

for row in edges:
    mat[row[0] - 1][row[1] - 1] = 1

print(edges)
mat

[[1, 2], [2, 1], [2, 3], [3, 2], [3, 4], [4, 3]]


[[0, 1, 0, 0], [1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0]]

In [77]:
ones = []
for row in mat:
    y = 0
    for elem in row:
        if elem == 1:
            y = y + 1
    ones.append(y)
    
ones

[1, 2, 2, 1]

In [78]:
def gen_nodes(mat, edges):
    nodes = set()
    for row in edges:
        nodes.add(row[0])
        nodes.add(row[1])
    return sorted(nodes)

In [79]:
nodes = gen_nodes(mat, edges)
nodes

[1, 2, 3, 4]

# Generating Probability Transition Matrix

In [80]:
def gen_ptm(alpha, ones):
    tel_ptm = [[0 for i in range(cols)] for j in range(rows)]
    nontel_ptm = [[0 for i in range(cols)] for j in range(rows)]
    
    for a in range(num):
        for b in range(num):
            if mat[a][b] == 1:
                tel_ptm[a][b] = (1 - alpha)/ones[a]
                nontel_ptm[a][b] = 1/ones[a]
            else:
                tel_ptm[a][b] = alpha/(num - ones[a])
                
    return tel_ptm, nontel_ptm

In [81]:
tel_ptm, nontel_ptm = gen_ptm(0.1, ones)
tel_ptm

[[0.03333333333333333, 0.9, 0.03333333333333333, 0.03333333333333333],
 [0.45, 0.05, 0.45, 0.05],
 [0.05, 0.45, 0.05, 0.45],
 [0.03333333333333333, 0.03333333333333333, 0.9, 0.03333333333333333]]

# Left Principle Eigenvalue Vector Method

In [82]:
def gen_pagerank(mat):
    np_mat = np.array(mat)
    v, V = np.linalg.eig(np_mat.T)
    score = V[:, v.argmax()].T
    score = score / score.sum()
    score = score.real
    return score

In [83]:
lev_tel = gen_pagerank(tel_ptm)
lev_nontel = gen_pagerank(nontel_ptm)

In [84]:
lev_map = {}
for x in range(len(nodes)):
    lev_map[nodes[x]] = round(lev_tel[x], 3)

lev_map_sorted = sorted(lev_map.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)
lev_map_sorted

[(3, 0.326), (2, 0.326), (4, 0.174), (1, 0.174)]

In [85]:
nonlev_map = {}
for x in range(len(nodes)):
    nonlev_map[nodes[x]] = round(lev_nontel[x], 3)

nonlev_map_sorted = sorted(nonlev_map.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)
nonlev_map_sorted

[(3, 0.333), (2, 0.333), (4, 0.167), (1, 0.167)]

# Power Iteration Method

In [86]:
def abs_diff(state, old_state):
    delta = state - old_state
    return math.sqrt(delta.dot(delta))

In [87]:
def initial_state(num):
    initial_prob = 1.0 / num
    start = []
    for x in range(num):
        start.append(initial_prob)
    return pd.Series(start)

In [88]:
def power_iteration(prob_mat, epsilon=0.00001, max_iter=1000):

    prob_mat = pd.DataFrame(prob_mat)
    state = initial_state(num)

    # Power iteration:
    for x in range(max_iter):
        prev_state = state.copy()
        state = state.dot(prob_mat)
        if abs_diff(state, prev_state) < epsilon:
            break

    return state

In [89]:
mat1 = power_iteration(tel_ptm)
mat2 = power_iteration(nontel_ptm)

In [90]:
levpi_map = {}
for x in range(len(nodes)):
    levpi_map[nodes[x]] = round(mat1[x], 3)

levpi_map_sorted = sorted(levpi_map.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)
levpi_map_sorted

[(3, 0.326), (2, 0.326), (4, 0.174), (1, 0.174)]

In [91]:
non_levpi_map = {}
for x in range(len(nodes)):
    non_levpi_map[nodes[x]] = round(mat2[x], 3)

non_levpi_map_sorted = sorted(non_levpi_map.items(), key = lambda kv:(kv[1], kv[0]), reverse = True)
non_levpi_map_sorted

[(3, 0.333), (2, 0.333), (4, 0.167), (1, 0.167)]